In [1]:

import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), r'C:\Users\Joel\Desktop\Sprints Case Study'))

if project_root not in sys.path:
    sys.path.append(project_root)

from utils import api_client

base_url = "https://technical-case-platform-engineer.onrender.com"

In [2]:
connect = api_client(base_url)

data_extract = connect.get("monthly-data")
currencies = connect.get("exchange-rates")

TypeError: 'module' object is not callable